In [1]:
import glob
import pickle
import os
import sys

if not os.path.join(os.path.abspath(''), '../') in sys.path:
    sys.path.append(os.path.join(os.path.abspath(''), '../'))
    
import numpy as np
import pandas as pd
from tqdm import notebook

from utils.feature_utils import filter_pose_noise

### specific to where I stored the csvs. you can change the data_root_dir

In [2]:
data_root_dir = '/Users/alexanderhsu/Documents/ACC_Study5_Fig4'
drugs = ['Morphine', 'Morphine_Formalin']
groups = ['Male', 'Female', 'Combined']
conditions = ['Control', 'Experimental']

### store all data in dictionary

In [3]:
filenames = {key: [] for key in drugs}
data = {key: [] for key in drugs}
for drug in notebook.tqdm(drugs):
    filenames[drug] = {key: [] for key in groups}
    data[drug] = {key: [] for key in groups}
    for group in notebook.tqdm(groups):
        filenames[drug][group] = {key: [] for key in conditions}
        data[drug][group] = {key: [] for key in conditions}
        for condition in notebook.tqdm(conditions):
            
            filenames[drug][group][condition] = glob.glob(str.join('/', 
                                                                  (data_root_dir,
                                                                   f'{drug}',
                                                                   f'{group}', 
                                                                   f'{condition}', 
                                                                   '*.csv')))
            data[drug][group][condition] = {key: [] for key in [f'file{i}' 
                                                               for i in range(len(filenames[drug][group][condition]))]}
            for i, csv in enumerate(notebook.tqdm(filenames[drug][group][condition])):
                temp_df = pd.read_csv(csv, header=[0, 1, 2, 3], sep=",", index_col=0)
                selected_pose_idx = np.arange(temp_df.shape[1])
                idx_llh = selected_pose_idx[2::3]
                # the loaded sleap file has them too, so exclude for both
                idx_selected = [i for i in selected_pose_idx if i not in idx_llh]
                currdf_filt, _ = filter_pose_noise(temp_df, idx_selected=idx_selected, idx_llh=idx_llh, 
                                                   llh_value=0.1)
                data[drug][group][condition][f'file{i}'] = currdf_filt

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
file_chosen = 0
filenames['Morphine']['Combined']['Control'][file_chosen]

In [2]:
data['Morphine_Formalin']['Female']['Experimental'][f'file{file_chosen}']

NameError: name 'data' is not defined

In [7]:
with open('../processed_dataset/raw_data_study5.pkl', 'wb') as f:
    pickle.dump(data, f)